# Práctica 0. Nódos y Tópicos

## Objetivo

Que los alumnos comprendan los conceptos básicos relacionados con los Nodos y los Tópicos, mediante la creación de un paquete con políticas en Python.

## Previo de la Práctica 0

### Definición de Nodo y Tópico en ROS 2

**Nodo**
- Un **nodo** es la unidad fundamental de computación en ROS 2. Cada nodo es un proceso ejecutable (programa) que realiza una tarea específica dentro del sistema robótico.

**Características principales:**
- Funcionalidad especializada: Cada nodo suele encargarse de una única función (ej: leer sensores, controlar motores, planificar rutas)
- Comunicación: Los nodos se comunican entre sí mediante mensajes, usando mecanismos como tópicos, servicios o acciones
- Independencia: Son entidades independientes que pueden ejecutarse en la misma máquina o distribuirse en diferentes dispositivos
- Ejemplo: Un nodo para leer datos de una cámara, otro para procesar esas imágenes, y otro para controlar los movimientos del robot

**Tópico**
- Un **tópico** es un canal de comunicación asíncrono y unidireccional mediante el cual los nodos intercambian mensajes usando el patrón publicador-suscriptor.

**Características principales:**
- Comunicación por mensajes: Los nodos publican mensajes en un tópico o se suscriben a él para recibirlos
- Desacoplado: Los publicadores y suscriptores no necesitan conocerse entre sí, solo necesitan saber el nombre del tópico
- Asíncrono: La comunicación no requiere que emisor y receptor estén activos al mismo tiempo
- Ejemplo: Un nodo publica lecturas de un sensor láser en el tópico `/scan`, mientras otro nodo se suscribe a ese tópico para recibir esas lecturas y evitar obstáculos

### Los tres tipos de interfaces que maneja ROS 2

**1. Mensajes (Messages)**
- **Comunicación unidireccional y asíncrona** mediante el patrón publicador-suscriptor
- **Flujo de datos continuo** ideal para sensores, telemetría y estados variables
- **Ejemplo:** Datos de láser, imágenes de cámara, posición del robot
- **No hay confirmación de recepción** ni garantía de procesamiento

**2. Servicios (Services)**
- **Comunicación bidireccional y síncrona** mediante el patrón cliente-servidor
- **Solicitud-respuesta** donde el cliente espera una respuesta del servidor
- **Ejemplo:** Comandos específicos, cálculos bajo demanda, consultas puntuales
- **Conexión directa** con confirmación inmediata de éxito/error

**3. Acciones (Actions)**
- **Comunicación bidireccional y asíncrona** para operaciones de larga duración
- **Patrón cliente-servidor con feedback** intermedio y capacidad de cancelación
- **Ejemplo:** Navegación a un punto, manipulación de objetos, misiones complejas
- **Permite monitorear progreso** y abortar la operación si es necesario

### ¿Cómo se documenta la creación de paquetes en programación (árbol)?

La documentación de paquetes en programación utiliza representaciones en árbol que muestran jerarquías de directorios con símbolos visuales intuitivos (📁 carpetas, 📄 archivos), pero su verdadero valor está en los metadatos técnicos que incluye: dependencias específicas con versiones, scripts de instalación, configuraciones de compilación, parámetros de despliegue, estructuras de módulos importables y políticas de mantenimiento. Esta documentación no solo muestra la estructura, sino que define relaciones críticas entre componentes, especifica entornos de ejecución, establece protocolos de testing y documenta APIs, convirtiéndose en el blueprint técnico esencial para la instalación, distribución y escalabilidad del software en diferentes entornos.

## Desarrollo de la Práctica 0

### Código nodo publicador:

In [ ]:
#!/usr/bin/env python3

import rclpy
from rclpy.node import Node
from std_msgs.msg import Float64
from math import sin, pi
import time

class MyPublisher(Node):
    def __init__(self):
        super().__init__("publish_node")
        self.publisher_ = self.create_publisher(Float64, "publish_topic", 10)
        self.get_logger().info("Node publicador senoidal activo")
        self.start_time = time.time()
        self.create_timer(0.1, self.timer_callback)  # ejecuta cada 0.1 segundos

    def timer_callback(self):
        t = time.time() - self.start_time  # tiempo transcurrido
        rpm = 1500 + 500 * sin(2 * pi * 0.5 * t)  # señal senoidal en RPM
        msg = Float64()
        msg.data = rpm
        self.publisher_.publish(msg)
        self.get_logger().info(f"Publicando: {rpm:.2f} RPM")

def main(args=None):
    rclpy.init(args=args)
    node = MyPublisher()
    rclpy.spin(node)
    rclpy.shutdown()

if __name__ == '__main__':
    main()

**Explicación:** Este nodo ROS 2 funciona como un generador de señales senoidales que publica valores de RPM (Revoluciones Por Minuto) en el tópico "publish_topic", simulando el comportamiento de un sensor de motor que oscila entre 1000 y 2000 RPM con una frecuencia de 0.5 Hz, publicando nuevos valores cada 0.1 segundos para que otros nodos del sistema puedan suscribirse y utilizar estos datos en tiempo real.

### Código nodo subscriptor-publicador

In [ ]:
#!/usr/bin/env python3

import rclpy
from rclpy.node import Node
from std_msgs.msg import Float64
from math import pi

class SubNode(Node):
    def __init__(self):
        super().__init__("node_sub_pub")

        # Suscriptor al tópico de RPM (float)
        self.subscriber_ = self.create_subscription(
            Float64,
            "publish_topic",
            self.sub_callback,
            10
        )

        # Publicador al nuevo tópico en rad/s
        self.publisher_ = self.create_publisher(
            Float64,
            "rad_vel_topic",
            10
        )
        self.get_logger().info("Modo subscriptor-publicador activo (RPM → rad/s)")

    def sub_callback(self, msg):
        rpm = msg.data
        rad_s = rpm * 2 * pi / 60.0  # Conversión: RPM → rad/s

        new_msg = Float64()
        new_msg.data = rad_s

        # Publica el resultado
        self.publisher_.publish(new_msg)
        self.get_logger().info(f"RPM: {rpm:.2f} → RAD/S: {rad_s:.2f}")

def main(args=None):
    rclpy.init(args=args)
    node = SubNode()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()

if __name__ == "__main__":
    main()

**Explicación:** Este nodo ROS 2 actúa como un convertidor en tiempo real que se suscribe al tópico "publish_topic" para recibir valores en RPM, los convierte a radianes por segundo usando la fórmula rad_s = RPM × 2π / 60, y publica el resultado convertido en el tópico "rad_vel_topic", permitiendo que otros componentes del sistema robótico trabajen con unidades angulares estándar.

## Resultados

### Videos:

Link del vídeo: https://youtu.be/P5jUkIkD17w

## Conclusiones

Basándome en lo propuesto en los objetivos puedo decir que se cumplieron con los mismos, ya que gracias a la construcción de los diferentes nodos usados durante la clase y esta práctica, logré una mejor comprensión del uso y la diferencia que hay entre un nodo subscriptor, publicador y subscriptor-publicador. Nos ayudan a determinar la arquitectura, eficiencia y confiabilidad del sistema robótico, ya que cada patrón de comunicación (mensajes, servicios o acciones) resuelve problemas específicos.

## Bibliografía

- S. Macenski, T. Foote, B. Gerkey, C. Lalancette, and W. Woodall, "Robot Operating System 2: Design, architecture, and uses in the wild," Science Robotics, vol. 7, no. 66, p. eabm6074, May 2022.

- D. H. Moon, S. H. Lee, and J. H. Park, "Design and implementation of ROS 2 based robot system architecture for autonomous driving," in 2021 21st International Conference on Control, Automation and Systems (ICCAS), Jeju, Korea, 2021, pp. 1206-1209.

- M. A. Karkoum, A. A. Makarem, and M. N. Kabalan, "Real-time performance evaluation of ROS 2 for robotic applications," in 2019 International Conference on Electrical and Computing Technologies and Applications (ICECTA), Ras Al Khaimah, UAE, 2019, pp. 1-5.

- Y. Maruyama, S. Kato, and T. Azumi, "Exploring the performance of ROS2," in 2016 International Conference on Embedded Software (EMSOFT), Pittsburgh, PA, USA, 2016, pp. 1-10.